## Statistical Modeling and Performance Evaluation<a href="#Statistical-Modeling-and-Performance-Evaluation" class="anchor-link"></a>

### Full Model<a href="#Full-Model" class="anchor-link"></a>

We begin by fitting a multiple linear regression that predicts `age`
using all of the available features. We call this the full model. First
let's take a quick peak at the clean data.

In [ ]:
data.head()

When constructing the regression formula, we can manually add all the
independent features. On the other hand, if there are lots of
independent variables, we can get smart and use some string function
tricks as below.

In [ ]:
# short and sweet
formula_string_indep_vars = ' + '.join(data.drop(columns='age').columns)
formula_string = 'age ~ ' + formula_string_indep_vars
print('formula_string: ', formula_string)

The formula string above works just fine with the `Statsmodels` module.
The problem, however, is that we cannot do automatic variable selection
with this formula. What we need for this purpose is "one-hot-encoding"
of categorical features. For more information on this encoding, please
refer to [this
page](https://www.featureranking.com/tutorials/machine-learning-tutorials/data-preparation-for-machine-learning/).

In the code chunk below, we first use the `get_dummies()` function in
`Pandas` for one-hot-encoding of categorical features and then we
construct a new formula string with the encoded features.

In [ ]:
# one-hot-encoding of categorical features
# for this to work correctly, variable data types (numeric or categorical)
# must be correctly specified within the Pandas dataframe
data_encoded = pd.get_dummies(data, drop_first=True)
data_encoded.head()

In [ ]:
formula_string_indep_vars_encoded = ' + '.join(data_encoded.drop(columns='age').columns)
formula_string_encoded = 'age ~ ' + formula_string_indep_vars_encoded
print('formula_string_encoded: ', formula_string_encoded)

For fun, let's add two interaction terms to our full model. Let's add
the interaction of the `capital` feature with `hours_per_week` and
`race_White` respectively.

In [ ]:
formula_string_encoded = formula_string_encoded + ' + hours_per_week:capital + race_White:capital'

Also, let's add the square of the `hours_per_week` feature to illustrate
how we can add higher order terms to our linear regression.

In [ ]:
formula_string_encoded = formula_string_encoded + ' + np.power(hours_per_week, 2)'
print('formula_string_encoded: ', formula_string_encoded)

Now that we have defined our statistical model formula as a Python
string, we fit an OLS (ordinary least squares) model to our encoded
data.

In [ ]:
model_full = sm.formula.ols(formula=formula_string_encoded, data=data_encoded)
###
model_full_fitted = model_full.fit()
###
print(model_full_fitted.summary())

The full model has an adjusted R-squared value of 0.405, which means
that only 40% of the variance is explained by the model. By looking at
the p-values, we observe that the majority of them are highly
significant, though there are a few insignificant variables at a 5%
level.

Let's define a new data frame for actual age vs. predicted age and the
residuals for the full model. We will use this data frame when plotting
predicted values and the regression residuals.

In [ ]:
residuals_full = pd.DataFrame({'actual': data_encoded['age'], 
                               'predicted': model_full_fitted.fittedvalues, 
                               'residual': model_full_fitted.resid})
residuals_full.head(10)

Let's plot actual age values vs. predicted values.

In [ ]:
def plot_line(axis, slope, intercept, **kargs):
    xmin, xmax = axis.get_xlim()
    plt.plot([xmin, xmax], [xmin*slope+intercept, xmax*slope+intercept], **kargs)

# Creating scatter plot
plt.scatter(residuals_full['actual'], residuals_full['predicted'], alpha=0.3);
plot_line(axis=plt.gca(), slope=1, intercept=0, c="red");
plt.xlabel('Actual Age');
plt.ylabel('Predicted Age');
plt.title('Figure 9: Scatter plot of actual vs. predicted age for the Full Model', fontsize=15);
plt.show();

From Figure 9, we observe that the model never produces a prediction
above 80 even though the oldest person in the dataset is 90.

We will now check the diagnostics for the full model.

### Full Model Diagnostic Checks<a href="#Full-Model-Diagnostic-Checks" class="anchor-link"></a>

We would like to check whether there are indications of violations of
the regression assumptions, which are

1.  linearity of the relationship between target variable and the
    independent variables
2.  constant variance of the errors
3.  normality of the residual distribution
4.  statistical independence of the residuals

Let's first get a scatter plot of residuals (as a function of predicted
`age`).

In [ ]:
plt.scatter(residuals_full['predicted'], residuals_full['residual'], alpha=0.3);
plt.xlabel('Predicted Age');
plt.ylabel('Residuals')
plt.title('Figure 10(a): Scatterplot of residuals vs. predicted age for Full Model', fontsize=15)
plt.show();

From Figure 10(a), we see that, rather than being mostly random and
centered around 0, the residuals exhibit a banding pattern, especially
when predicted age is below 50. This pattern indicates that the constant
variability assumption of linear regression is not quite satisfied in
this case.

Let's now plot actual age vs. residuals.

In [ ]:
plt.scatter(residuals_full['actual'], residuals_full['residual'], alpha=0.3);
plt.xlabel('Actual Age');
plt.ylabel('Residuals')
plt.title('Figure 10(b): Scatterplot of residuals vs. actual age for Full Model', fontsize=15)
plt.show();

We notice that the model overestimates younger ages and underestimates
older ages. In particular, for those younger than the age of 30, the
model predicts much older ages. Also, for those above the age of 80, the
model predicts significantly younger ages.

Let's overlay the histograms of actual vs. predicted age on the same
plot.

In [ ]:
plt.hist(residuals_full['actual'], label='actual', bins=20, alpha=0.7);
plt.hist(residuals_full['predicted'], label='predicted', bins=20, alpha=0.7);
plt.xlabel('Age');
plt.ylabel('Frequency');
plt.title('Figure 11: Histograms of actual age vs. predicted age for Full Model', fontsize=15);
plt.legend()
plt.show();

We notice that their distributions are quite different. In particular,
the model's predictions are highly clustered around mid-40's.

Let's now have look at the histogram of the residuals.

In [ ]:
plt.hist(residuals_full['residual'], bins = 20);
plt.xlabel('Residual');
plt.ylabel('Frequency');
plt.title('Figure 12: Histogram of residuals for Full Model', fontsize=15);
plt.show();

From Figure 12, the histogram of residuals looks somewhat symmetric,
though slightly right-skewed. Nonetheless, it seems the normality
assumption of linear regression is not significantly violated in this
particular case.

### Backwards Feature Selection<a href="#Backwards-Feature-Selection" class="anchor-link"></a>

We now perform backwards feature selection using p-values. It appears
`Statsmodels` does not have any canned code for automatic feature
selection, so we wrote one ourselves.

In [ ]:
## create the patsy model description from formula
patsy_description = patsy.ModelDesc.from_formula(formula_string_encoded)

# initialize feature-selected fit to full model
linreg_fit = model_full_fitted

# do backwards elimination using p-values
p_val_cutoff = 0.05

## WARNING 1: The code below assumes that the Intercept term is present in the model.
## WARNING 2: It will work only with main effects and two-way interactions, if any.

print('\nPerforming backwards feature selection using p-values:')

while True:

    # uncomment the line below if you would like to see the regression summary
    # in each step:
    ### print(linreg_fit.summary())

    pval_series = linreg_fit.pvalues.drop(labels='Intercept')
    pval_series = pval_series.sort_values(ascending=False)
    term = pval_series.index[0]
    pval = pval_series[0]
    if (pval < p_val_cutoff):
        break
    term_components = term.split(':')
    print(f'\nRemoving term "{term}" with p-value {pval:.4}')
    if (len(term_components) == 1): ## this is a main effect term
        patsy_description.rhs_termlist.remove(patsy.Term([patsy.EvalFactor(term_components[0])]))    
    else: ## this is an interaction term
        patsy_description.rhs_termlist.remove(patsy.Term([patsy.EvalFactor(term_components[0]), 
                                                        patsy.EvalFactor(term_components[1])]))    

    linreg_fit = smf.ols(formula=patsy_description, data=data_encoded).fit()

###
## this is the clean fit after backwards elimination
model_reduced_fitted = smf.ols(formula = patsy_description, data = data_encoded).fit()
###

#########
print("\n***")
print(model_reduced_fitted.summary())
print("***")
print(f"Regression number of terms: {len(model_reduced_fitted.model.exog_names)}")
print(f"Regression F-distribution p-value: {model_reduced_fitted.f_pvalue:.4f}")
print(f"Regression R-squared: {model_reduced_fitted.rsquared:.4f}")
print(f"Regression Adjusted R-squared: {model_reduced_fitted.rsquared_adj:.4f}")

Similar to what we did for the full model, let's define a new data frame
for actual age vs. predicted age and the residuals for the reduced
model.

In [ ]:
residuals_reduced = pd.DataFrame({'actual': data_encoded['age'], 
                                  'predicted': model_reduced_fitted.fittedvalues, 
                                  'residual': model_reduced_fitted.resid})
residuals_reduced.head(10)

In [ ]:
# get a scatter plot
plt.scatter(residuals_reduced['actual'], residuals_reduced['predicted'], alpha=0.3);
plot_line(axis=plt.gca(), slope=1, intercept=0, c="red");
plt.xlabel('Age');
plt.ylabel('Predicted Age');
plt.title('Figure 13: Scatter plot of actual vs. predicted age for Reduced Model', fontsize=15);
plt.show(); 

This model returns an Adjusted R-squared of 0.404, meaning the reduced
model still explains about 40% of the variance, but with 6 less
variables. Looking at the p-values, they are all significant at the 5%
level, as expected. From Figure 13, we still have the same issues with
our model. That is, the model overestimates younger ages and
underestimates older ages. We will now perform the diagnostic checks on
this reduced model.

### Reduced Model Diagnostic Checks<a href="#Reduced-Model-Diagnostic-Checks" class="anchor-link"></a>

Let's first get a scatter plot of residuals (as a function of predicted
age).

In [ ]:
plt.scatter(residuals_reduced['predicted'], residuals_reduced['residual'], alpha=0.3);
plt.xlabel('Predicted Age');
plt.ylabel('Residuals')
plt.title('Figure 14: Scatter plot of residuals vs. predicted age for Reduced Model', fontsize=15)
plt.show();

Figure 14 looks very similar to Figure 10(a), suggesting that the
residuals exhibit the same banding pattern.

Let's now have look at the histogram of the residuals for the reduced
model.

In [ ]:
plt.hist(residuals_reduced['residual'], bins = 20);
plt.xlabel('Residual');
plt.ylabel('Frequency');
plt.title('Figure 15: Histogram of residuals for Reduced Model', fontsize = 15)
plt.show();

From Figure 15, there is again a somewhat symmetric histogram around
zero, which suggests that the residuals are somewhat normally
distributed.

## Summary and Conclusions<a href="#Summary-and-Conclusions" class="anchor-link"></a>

Using our independent variables, we were able to get a full model with
an Adjusted R-squared value of about 40%. After backwards variable
selection with a p-value cutoff value of 0.05, we were able to maintain
the same performance but with 6 less variables. Our final model has 49
variables all together with a model p-value of 0.

Diagnostic checks with residual scatter plots indicate that, rather than
being random and centered around 0, the residuals exhibit a banding
pattern, especially when predicted age is below 50. This pattern
indicates that the constant variability assumption of linear regression
is not quite satisfied in this case. On the other hand, residual
histograms suggest that there are no significant violations of the
normality assumption on the residuals.

The final multiple linear regression model has an Adjusted R-squared
value of about 40%, which is pretty low. So, it appears that the
variables we used are not quite adequate for accurately predicting the
age of an individual in the 1994 US Census dataset within a multiple
linear regression framework. A good next step might involve adding some
more interaction terms and maybe some other higher order terms to see if
this would result in some improvement for the Adjusted R-squared value.
Nonetheless, it might be the case that nonlinear models such as a neural
network might be more appropriate for the task at hand rather than a
linear regression model.

Our regression model appears to predict age correctly within \$\\pm40\$
years in general, though this is clearly a huge margin of error for the
model to be useful for any practical purposes. Furthermore, our model
has some rather significant issues. Specifically, our model consistently
overestimates younger ages and underestimates older ages. In particular,
for those younger than the age of 30, the model predicts much older
ages. Also, for those above the age of 80, the model predicts
significantly younger ages.


***
featureranking.com